In [3]:
import cv2
import os
import numpy as np
from skimage.color import rgb2hsv
from skimage import filters
from skimage.exposure import equalize_hist

def enhance_contrast(image):
    # 對比增強
    return cv2.equalizeHist(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))

def rgb_to_hsv(image):
    # RGB轉HSI/HSV
    hsv_image = rgb2hsv(image)
    intensity = hsv_image[:, :, 2]  # 提取亮度通道
    return intensity

def calculate_otsu_threshold(image):
    # 計算 Otsu 閾值
    return filters.threshold_otsu(image)

def apply_joint_threshold(image, otsu_thresh):
    # 計算聯合閾值
    _, binary_global = cv2.threshold(image, otsu_thresh, 255, cv2.THRESH_BINARY)
    return binary_global

def canny_edge_detection(image):
    # Canny 邊緣檢測
    return cv2.Canny(image, 100, 200)

def morphological_operations(image):
    # 形態學操作
    kernel = np.ones((5, 5), np.uint8)
    closed_image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    return closed_image

def process_image(image_path, output_path):
    # 讀取圖像
    image = cv2.imread(image_path)

    # 步驟 1: 對比增強
    enhanced_image = enhance_contrast(image)

    # 步驟 2: RGB 轉 HSI/HSV
    intensity = rgb_to_hsv(image)

    # 步驟 3: 計算 Otsu 閾值
    otsu_thresh = calculate_otsu_threshold(intensity)

    # 步驟 4: 計算聯合閾值
    joint_thresholded = apply_joint_threshold(enhanced_image, otsu_thresh)

    # 步驟 5: Canny 邊緣檢測
    edges = canny_edge_detection(joint_thresholded)

    # 步驟 6: 形態學操作
    final_image = morphological_operations(edges)

    # 保存結果
    cv2.imwrite(output_path, final_image)

def process_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
            process_image(input_path, output_path)
            #print(f"Processed: {filename}")

# 替換為您的資料夾路徑
input_folder = "/home/yuchi/Canny_Detection/crackforest/Images"
output_folder = "/home/yuchi/Canny_Detection/crackforest/Output_1"
process_folder(input_folder, output_folder)

In [2]:
import os
import cv2
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

def read_images_with_matching(folder, prefix, file_extension):
    """读取指定资料夹中指定格式的图片，并匹配文件名前缀"""
    image_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(file_extension)]
    images = {}
    for file in image_files:
        # 获取不含后缀的文件名，并去掉可能的 '_label'
        image_name = os.path.splitext(os.path.basename(file))[0].replace('_label', '')
        image = cv2.imread(file, cv2.IMREAD_GRAYSCALE)  # 读取灰度图像
        if image is not None:
            images[image_name] = image
        else:
            print(f"Warning: Could not read {file}")
    return images

def calculate_metrics(true_images, predicted_images, threshold=128):
    """计算 Precision, Recall 和 F-Measure"""
    precision_list, recall_list, f1_list = [], [], []
    
    for name, true_image in true_images.items():
        if name in predicted_images:
            pred_image = predicted_images[name]
            
            # 确保两张图片尺寸一致
            if true_image.shape != pred_image.shape:
                print(f"Skipping {name}: Shape mismatch ({true_image.shape} vs {pred_image.shape})")
                continue
            
            # 二值化处理
            true_binary = (true_image > threshold).astype(np.uint8).flatten()
            pred_binary = (pred_image > threshold).astype(np.uint8).flatten()
            
            # 计算指标
            precision = precision_score(true_binary, pred_binary, zero_division=0)
            recall = recall_score(true_binary, pred_binary, zero_division=0)
            f1 = f1_score(true_binary, pred_binary, zero_division=0)
            
            precision_list.append(precision)
            recall_list.append(recall)
            f1_list.append(f1)
        else:
            print(f"Warning: Missing matching image for {name} in predicted folder")
    
    # 计算平均指标
    avg_precision = np.mean(precision_list) if precision_list else 0
    avg_recall = np.mean(recall_list) if recall_list else 0
    avg_f1 = np.mean(f1_list) if f1_list else 0
    
    return avg_precision, avg_recall, avg_f1

def main(jpg_folder, png_folder, threshold=128):
    """主函数"""
    # 读取两个文件夹中的图片
    print("Reading .jpg images...")
    true_images = read_images_with_matching(jpg_folder, '', '.jpg')
    
    print("Reading .PNG images...")
    predicted_images = read_images_with_matching(png_folder, '_label', '.PNG')
    
    # 计算 Precision, Recall 和 F-Measure
    print("Calculating metrics...")
    precision, recall, f1 = calculate_metrics(true_images, predicted_images, threshold)
    
    # 输出结果
    print("\nResults:")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F-Measure: {f1:.4f}")

# 示例运行
if __name__ == "__main__":
    jpg_folder = "/home/yuchi/Canny_Detection/crackforest/Output_1"  # 替换为您的 .jpg 图片资料夹路径
    png_folder = "/home/yuchi/Canny_Detection/crackforest/Masks"  # 替换为您的 .PNG 图片资料夹路径
    main(jpg_folder, png_folder, threshold=128)


Reading .jpg images...
Reading .PNG images...
Calculating metrics...

Results:
Precision: 0.2789
Recall: 0.5391
F-Measure: 0.3522
